In [1]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2021
mmdd_str = today.strftime('%m%d')
mmdd_str

'0217'

In [2]:
#today = date(2022, 2, 15)
mmdd_str = today.strftime('%m%d')
mmdd_str

'0217'

In [3]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  32


In [4]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  DOHOME
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517723590
Published:  Thu, 17 Feb 2022 21:07:40 +0700

----------------------------------

F45: 1
Title:  SMT
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721500
Published:  Thu, 17 Feb 2022 20:12:28 +0700

----------------------------------

F45: 2
Title:  TPIPP
Year:  2021
Quarter:  S
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721480
Published:  Thu, 17 Feb 2022 20:12:13 +0700

----------------------------------

F45: 3
Title:  BCP
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721180
Published:  Thu, 17 Feb 2022 20:08:20 +0700

----------------------------------

F45: 4
Title:  STGT
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.d

In [5]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']]

,name,year,quarter,published
0,DOHOME,2021,y,"Thu, 17 Feb 2022 21:07:40 +0700"
1,SMT,2021,y,"Thu, 17 Feb 2022 20:12:28 +0700"
2,TPIPP,2021,S,"Thu, 17 Feb 2022 20:12:13 +0700"
3,BCP,2021,y,"Thu, 17 Feb 2022 20:08:20 +0700"
4,STGT,2021,y,"Thu, 17 Feb 2022 19:47:19 +0700"
5,TTLPF,2021,e,"Thu, 17 Feb 2022 19:45:57 +0700"
6,STA,2021,y,"Thu, 17 Feb 2022 19:44:06 +0700"
7,ASEFA,2021,y,"Thu, 17 Feb 2022 19:11:32 +0700"
8,PTT,2021,y,"Thu, 17 Feb 2022 18:42:43 +0700"
9,BCP,2021,y,"Thu, 17 Feb 2022 18:38:17 +0700"


In [7]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == 'y','quarter'] = 4
#df.replace({"quarter" : "1"}, "3", inplace=True)
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,30,30,30,30
S,1,1,1,1
e,1,1,1,1


In [8]:
df.loc[df.quarter == 'e','quarter'] = 4
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,31,31,31,31
S,1,1,1,1


In [9]:
df.loc[df.quarter == 'S','quarter'] = 4
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,32,32,32,32


In [10]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == '1','year'] = 2022
#df.replace({"quarter" : "1"}, "3", inplace=True)
df.groupby(['year']).count()

,name,quarter,link,published
year,,,,
2021,32,32,32,32


In [11]:
df.shape

(32, 5)

In [12]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df.shape

(29, 5)

In [13]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(one_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [14]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

299

In [15]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(320, 7)

### Tickers that won't be input

In [16]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
]
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
7,ASEFA,2021.0,4,"Thu, 17 Feb 2022 19:11:32 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517718690
10,MGT,2021.0,4,"Thu, 17 Feb 2022 18:09:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517711070
13,SPVI,2021.0,4,"Thu, 17 Feb 2022 17:44:16 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517706000
14,CPT,2021.0,4,"Thu, 17 Feb 2022 17:37:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517703980
18,ZIGA,2021.0,4,"Thu, 17 Feb 2022 13:10:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517681180
19,PM,2021.0,4,"Thu, 17 Feb 2022 12:33:02 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517679700
21,CHEWA,2021.0,4,"Thu, 17 Feb 2022 08:54:14 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517671400
24,TAE,2021.0,4,"Thu, 17 Feb 2022 08:26:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517669780


In [17]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

8

In [18]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,DOHOME,2021.0,4,"Thu, 17 Feb 2022 21:07:40 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517723590
1,SMT,2021.0,4,"Thu, 17 Feb 2022 20:12:28 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721500
2,TPIPP,2021.0,4,"Thu, 17 Feb 2022 20:12:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721480
3,BCP,2021.0,4,"Thu, 17 Feb 2022 20:08:20 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721180
4,STGT,2021.0,4,"Thu, 17 Feb 2022 19:47:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517720010
5,TTLPF,2021.0,4,"Thu, 17 Feb 2022 19:45:57 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719910
6,STA,2021.0,4,"Thu, 17 Feb 2022 19:44:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719830
8,PTT,2021.0,4,"Thu, 17 Feb 2022 18:42:43 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517715190
9,SC,2021.0,4,"Thu, 17 Feb 2022 18:28:17 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517713530
11,TFM,2021.0,4,"Thu, 17 Feb 2022 17:58:44 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517709670


In [19]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

21

In [20]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(403, 9)

In [21]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(404, 14)

### There are no ticker records

In [22]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
]

,name,year,quarter,published,link,id,market
9,TFM,2021.0,4,"Thu, 17 Feb 2022 17:58:44 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517709670,NaN,NaN


### There are ticker records

In [23]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market
0,DOHOME,2021.0,4,"Thu, 17 Feb 2022 21:07:40 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517723590,701.0,SET100 / SETWB
1,SMT,2021.0,4,"Thu, 17 Feb 2022 20:12:28 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721500,456.0,sSET
2,TPIPP,2021.0,4,"Thu, 17 Feb 2022 20:12:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721480,560.0,SET
3,BCP,2021.0,4,"Thu, 17 Feb 2022 20:08:20 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721180,52.0,SET100 / SETCLMV / SETTHSI
4,STGT,2021.0,4,"Thu, 17 Feb 2022 19:47:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517720010,714.0,SET50 / SETTHSI
5,TTLPF,2021.0,4,"Thu, 17 Feb 2022 19:45:57 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719910,707.0,SET
6,STA,2021.0,4,"Thu, 17 Feb 2022 19:44:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719830,481.0,SET100 / SETTHSI / SETWB
7,PTT,2021.0,4,"Thu, 17 Feb 2022 18:42:43 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517715190,383.0,SET50 / SETCLMV / SETHD / SETTHSI
8,SC,2021.0,4,"Thu, 17 Feb 2022 18:28:17 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517713530,425.0,sSET / SETTHSI
10,MFC,2021.0,4,"Thu, 17 Feb 2022 17:56:42 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517708960,296.0,SET


In [24]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2

,name,year,quarter,published,link,id,market
0,DOHOME,2021.0,4,"Thu, 17 Feb 2022 21:07:40 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517723590,701.0,SET100 / SETWB
1,SMT,2021.0,4,"Thu, 17 Feb 2022 20:12:28 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721500,456.0,sSET
2,TPIPP,2021.0,4,"Thu, 17 Feb 2022 20:12:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721480,560.0,SET
3,BCP,2021.0,4,"Thu, 17 Feb 2022 20:08:20 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721180,52.0,SET100 / SETCLMV / SETTHSI
4,STGT,2021.0,4,"Thu, 17 Feb 2022 19:47:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517720010,714.0,SET50 / SETTHSI
5,TTLPF,2021.0,4,"Thu, 17 Feb 2022 19:45:57 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719910,707.0,SET
6,STA,2021.0,4,"Thu, 17 Feb 2022 19:44:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719830,481.0,SET100 / SETTHSI / SETWB
7,PTT,2021.0,4,"Thu, 17 Feb 2022 18:42:43 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517715190,383.0,SET50 / SETCLMV / SETHD / SETTHSI
8,SC,2021.0,4,"Thu, 17 Feb 2022 18:28:17 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517713530,425.0,sSET / SETTHSI
10,MFC,2021.0,4,"Thu, 17 Feb 2022 17:56:42 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517708960,296.0,SET


In [25]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2

,name,year,quarter,published,link,id,market
0,DOHOME,2021,4,"Thu, 17 Feb 2022 21:07:40 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517723590,701.0,SET100 / SETWB
1,SMT,2021,4,"Thu, 17 Feb 2022 20:12:28 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721500,456.0,sSET
2,TPIPP,2021,4,"Thu, 17 Feb 2022 20:12:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721480,560.0,SET
3,BCP,2021,4,"Thu, 17 Feb 2022 20:08:20 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721180,52.0,SET100 / SETCLMV / SETTHSI
4,STGT,2021,4,"Thu, 17 Feb 2022 19:47:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517720010,714.0,SET50 / SETTHSI
5,TTLPF,2021,4,"Thu, 17 Feb 2022 19:45:57 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719910,707.0,SET
6,STA,2021,4,"Thu, 17 Feb 2022 19:44:06 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719830,481.0,SET100 / SETTHSI / SETWB
7,PTT,2021,4,"Thu, 17 Feb 2022 18:42:43 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517715190,383.0,SET50 / SETCLMV / SETHD / SETTHSI
8,SC,2021,4,"Thu, 17 Feb 2022 18:28:17 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517713530,425.0,sSET / SETTHSI
10,MFC,2021,4,"Thu, 17 Feb 2022 17:56:42 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517708960,296.0,SET


In [26]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(one_file, header=True, index=False, sep=',')

In [27]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2021'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(1064, 14)

In [28]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(1083, 19)

### Already input, display profit amt & eps to check with new F45

In [29]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge
11,SCB,2021,4,"Thu, 17 Feb 2022 17:09:05 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517697950,426.0,SET50 / SETCLMV / SETTHSI,21088.0,7878555.0,4965445.0,35598816.0,27217602.0,2.32,1.46,10.47,8.01,426.0,2022-01-21,both


In [32]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3.shape

(19, 19)

In [33]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
inp3_str = df_inp3.name+' '+df_inp3.year+' '+df_inp3.quarter+' '+df_inp3.link
inp3_str

0     DOHOME 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517723590
1        SMT 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721500
2      TPIPP 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721480
3        BCP 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517721180
4       STGT 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517720010
5      TTLPF 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719910
6        STA 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517719830
7        PTT 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517715190
8         SC 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16450517713530
9        MFC 2021 4 https://www.set.or.th/set/